# DES
- DES是使用最广泛的**对称密码**：数据加密标准（Data Encryption Standard）。<br>
- 在对称密码系统（也叫做**私钥密码系统**）中，加密和解密使用**相同的密钥**。<br>
<div>
    <img align=left src="Images/des.png"/>
</div>

## 1. DES原理
### 1.1 基本工作原理
用56位的密钥对64位长的数据块进行16轮加密处理由此得到64位长的密文。
### 1.2 基本工作流程
<div>
    <img align=left src="Images/des-process.png"/>
</div>

## 2. 流程详解
在正式讲解如何把64位明文转换为64位密文之前，我们先讲一下DES加密使用的子密钥是如何生成的。
### 2.1 子密钥生成

子密钥生成的全流程如下：
<div>
    <img align=left src="Images/des-process-key-generate.png"/>
</div>

#### 2.1.1 输入**64位的密钥**到子密钥算法。

<div>
    <img align=left src="Images/key-generate-1.png"/>
</div>

#### 2.1.2 略去奇偶校验位
把每个字节的第8位作为奇偶校验位,并且略去。DES的密钥由64位减至56位：

<div>
    <img align=left src="Images/key-generate-2.png"/>
</div>

#### 2.1.3  置换选择1
把56位的密钥通过置换选择1，permutation Choice one (PC-1)：

<div>
    <img align=left src="Images/key-generate-3.png"/>
</div>

#### 2.1.4 密钥一分为二
把通过置换选择1的56位密钥，分成2部分（C，D），每部分有28位

<div>
    <img align=left src="Images/key-generate-4.png"/>
</div>

#### 2.1.5 循环左移
每轮中，这两部分分别**循环左移**l位或2位。下表给出了每轮移动的位数：

<div>
    <img align=left src="Images/key-generate-5.png"/>
</div>

#### 2.1.6 置换选择2
置换选择2,permutation choice two (PC-2)，也被称为：**压缩置换**。<br>
- 置换：例如，原第14位在输出时移到了第1位。
- 压缩：第9、18、22、25以及第35、38、43、54均被略去。

<div>
    <img align=left src="Images/key-generate-6.png"/>
</div>

经过置换选择2，我们终于实现了：将56位密钥压缩成48位。

## 明文变密文
以上就是子密钥的生成。接下来，我们开始进入正题——**把64位明文转换为64位密文**。<br>
如下图所示，要把64比特的明文转换成64比特的密文，需要经历：
1. 初始置换
2. 与子密钥K进行16轮迭代
3. 逆初始置换

<div>
    <img align=left src="Images/des-process-showtime.png"/>
</div>

### 2.2 初始置换
和子密钥的置换选择一样，64位的明文也要经历置换选择。

#### 2.2.1  把明文作为输入
初始置换把**64位**明文作为输入：

<div>
    <img align=left src="Images/IP1.png"/>
</div>

#### 2.2.2  置换明文
初始置换把明文按照原有的**64位**进行置换，最后还是**64位**：

<div>
    <img align=left src="Images/IP2.png"/>
</div>

#### 2.2.3  一分为二
把通过置换后的64位明文，分成2部分（L，R），每部分有32位

### 2.3 与子密钥K进行16轮迭代
通过2.2.3的一分为二，现在有L、R**各32位**的置换过的明文，我们拿出16轮中的一轮来看看到底这16轮迭代到底做了什么：

<div>
    <img align=left src="Images/des-encryption.jpg"/>
</div>

#### 2.3.1  扩展置换E
扩展置换E：指的是将原本的**32位块**（$R_{i-1}$）扩展到**48位**。

<div>
    <img align=left src="Images/EP1.png"/>
    <img align=left src="Images/E-Box.png"/>
</div>

将输入的**32位块**（$R_{i-1}$）扩展到**48位**的输出块。这样做的目的其实是实现和**子密钥K**进行**异或运算（XOR）**。<br>
PS：异或运算（XOR）要求同位之间进行，32位与48位是无法进行异或运算的。

#### 2.3.2  异或运算（XOR）
把通过**扩展置换E**扩展的**48位块**与**子密钥K**进行**异或运算（XOR）**

#### 2.3.3  压缩替换S
将**48位**压缩到**32位**。这样做的目的其实是实现和$L$**（32位块）**进行**异或运算（XOR）**。

####   逆置换
再将64位的明文进行**逆置换**：

<div>
    <img align=left src="Images/IP3.png"/>
</div>